In [11]:
!sudo apt-key adv --fetch-keys http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/7fa2af80.pub
!sudo sh -c 'echo "deb http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64 /" > /etc/apt/sources.list.d/cuda.list'
!sudo apt-get update && sudo apt-get install -y --no-install-recommends cuda-drivers

Ign http://archive.ubuntu.com trusty InRelease
Ign http://developer.download.nvidia.com  InRelease
Get:1 http://archive.ubuntu.com trusty-updates InRelease [65.9 kB]
Get:2 http://developer.download.nvidia.com  Release.gpg [819 B]
Get:3 http://developer.download.nvidia.com  Release [564 B]
Get:4 http://archive.ubuntu.com trusty-security InRelease [65.9 kB]
Get:5 http://archive.ubuntu.com trusty Release.gpg [933 B]
Get:6 http://developer.download.nvidia.com  Packages [22.5 kB]
Get:7 http://archive.ubuntu.com trusty Release [58.5 kB]
Get:8 http://archive.ubuntu.com trusty-updates/main Sources [482 kB]
Get:9 http://archive.ubuntu.com trusty-updates/restricted Sources [5957 B]
Get:10 http://archive.ubuntu.com trusty-updates/universe Sources [217 kB]
Get:11 http://archive.ubuntu.com trusty-updates/main amd64 Packages [1173 kB]
Get:12 http://archive.ubuntu.com trusty-updates/restricted amd64 Packages [20.4 kB]
Get:13 http://archive.ubuntu.com trusty-updates/universe amd64 Packages [511 kB]
Ge

In [1]:
!nvidia-smi

Mon Jan 30 01:48:11 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.57                 Driver Version: 367.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 0000:00:1E.0     Off |                    0 |
| N/A   40C    P8    30W / 149W |      0MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import numpy as np
import time
import sys
import math
from pprint import pprint
import theano
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [3]:
# test gpu
A = np.random.rand(1000,10000).astype(theano.config.floatX)
B = np.random.rand(10000,1000).astype(theano.config.floatX)
np_start = time.time()
AB = A.dot(B)
np_end = time.time()
X,Y = theano.tensor.matrices('XY')
mf = theano.function([X,Y],X.dot(Y))
t_start = time.time()
tAB = mf(A,B)
t_end = time.time()
print ("NP time: %f[s], theano time: %f[s] **(times should be close when run on CPU only)**" %
       (np_end-np_start, t_end-t_start))
print ("Result difference: %f" % (np.abs(AB-tAB).max(), ))

NP time: 0.577993[s], theano time: 0.298977[s] **(times should be close when run on CPU only)**
Result difference: 0.000000


In [4]:
# Create TensorFlow object called tensor
hello_constant = tf.constant('Hello World!')
with tf.Session() as sess:
    # Run the tf.constant operation in the session
    output = sess.run(hello_constant)
    print(output)

b'Hello World!'


In [5]:
# A is a 0-dimensional int32 tensor
A = tf.constant(1234)

# B is a 1-dimensional int32 tensor
B = tf.constant([ [123,456,789] ]) 

# C is a 2-dimensional int32 tensor
C = tf.constant([ [123,456,789], [222,333,444] ])

In [6]:
# constant tf
x = tf.placeholder(tf.string)
with tf.Session() as sess:
    output = sess.run(x, feed_dict={x: 'Hello World'})
    print(output)

Hello World


In [7]:
# basic tf
x = tf.placeholder(tf.string)
y = tf.placeholder(tf.int32)
z = tf.placeholder(tf.float32)

with tf.Session() as sess:
    output = sess.run(x, feed_dict={x: 'Test String', y: 123, z: 45.67})
    print(output)

Test String


In [8]:
# basic arith
x = 10
y = 2
z = x/y - 1
print(z)

4.0


In [9]:
# tf arith
x = tf.constant(10)
y = tf.constant(2)
z = tf.sub(tf.div(x,y), tf.constant(1))
with tf.Session() as sess:
    output = sess.run(z)
    print(output)

4


In [10]:
# simple mat alg
X = np.matrix('-0.5, 0.2, 0.1; 0.7, -0.8, 0.2')
W = np.vstack([0.2, 0.5, 0.6])
b = np.vstack([0.1, 0.2])
print(np.dot(X, W) + b)

[[ 0.16]
 [ 0.06]]


In [11]:
# set weights
n_features = 10000
n_labels = 10
bias = tf.Variable(tf.zeros(n_labels))
weights = tf.Variable(tf.truncated_normal((n_features, n_labels)))

# initialize a var
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

In [12]:
def mnist_features_labels(n_labels):
    """
    Gets the first <n> labels from the MNIST dataset
    :param n_labels: Number of labels to use
    :return: Tuple of feature list and label list
    """
    mnist_features = []
    mnist_labels = []

    mnist = input_data.read_data_sets('/datasets/ud730/mnist', one_hot=True)

    # In order to make quizzes run faster, we're only looking at 10000 images
    for mnist_feature, mnist_label in zip(*mnist.train.next_batch(55000)):

        # Add features and labels if it's for the first <n>th labels
        if mnist_label[:n_labels].any():
            mnist_features.append(mnist_feature)
            mnist_labels.append(mnist_label[:n_labels])

    return mnist_features, mnist_labels

In [13]:
def weights(n_features, n_labels):
    """
    Return TensorFlow weights
    :param n_features: Number of features
    :param n_labels: Number of labels
    :return: TensorFlow weights
    """
    return tf.Variable(tf.random_normal([n_features, n_labels]))

In [14]:
def biases(n_labels):
    """
    Return TensorFlow bias
    :param n_labels: Number of labels
    :return: TensorFlow bias
    """
    return tf.Variable(tf.random_normal([n_labels]))

In [15]:
def linear(input, w, b):
    """
    Return linear function in TensorFlow
    :param input: TensorFlow input
    :param w: TensorFlow weights
    :param b: TensorFlow biases
    :return: TensorFlow linear function
    """
    return tf.add(tf.matmul(input, w), b)

In [16]:
def softmax(x):
    """
    Compute softmax values for each sets of scores in x.
    """
    return np.exp(x)/np.sum(np.exp(x), axis=0)

In [17]:
# Number of features (28*28 image is 784 features)
n_features = 784

# Number of labels
n_labels = 10

# Features and Labels
features = tf.placeholder(tf.float32)
labels = tf.placeholder(tf.float32)

# Weights and Biases
w = weights(n_features, n_labels)
b = biases(n_labels)

# Linear Function xW + b
logits = linear(features, w, b)

# Training data
train_features, train_labels = mnist_features_labels(n_labels)

# init session
init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init)
    
    # Softmax
    prediction = tf.nn.softmax(logits)

    # Cross entropy
    cross_entropy = -tf.reduce_sum(labels * tf.log(prediction), reduction_indices=1)

    # Training loss
    loss = tf.reduce_mean(cross_entropy)

    # Rate at which the weights are changed
    learning_rate = 0.01

    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    # Run optimizer and get loss
    _, l = session.run(
        [optimizer, loss],
        feed_dict={features: train_features, labels: train_labels})

# Print loss
print('Loss: {}'.format(l))

Extracting /datasets/ud730/mnist/train-images-idx3-ubyte.gz
Extracting /datasets/ud730/mnist/train-labels-idx1-ubyte.gz
Extracting /datasets/ud730/mnist/t10k-images-idx3-ubyte.gz
Extracting /datasets/ud730/mnist/t10k-labels-idx1-ubyte.gz
Loss: 14.300590515136719


In [18]:
output = None
logit_data = [20.0, 10.0, 0.01]
logits = tf.placeholder(tf.float32)
softmax = tf.nn.softmax(logits)
with tf.Session() as sess:
    output = sess.run(softmax, feed_dict={logits:logit_data})
print(output)

[  9.99954581e-01   4.53978682e-05   2.08177431e-09]


In [19]:
softmax_data = [0.7, 0.2, 0.1]
one_hot_data = [1.0, 0.0, 0.0]
softmax = tf.placeholder(tf.float32)
one_hot = tf.placeholder(tf.float32)
cross_ent = -tf.reduce_sum(tf.mul(one_hot, tf.log(softmax)))
with tf.Session() as sess:
    print(sess.run(cross_ent, feed_dict={one_hot:one_hot_data, softmax:softmax_data}))

0.356675


In [21]:
# The features are already scaled and the data is shuffled
mnist = input_data.read_data_sets('/datasets/ud730/mnist', one_hot=True)
train_features = mnist.train.images
test_features = mnist.test.images
train_labels = mnist.train.labels.astype(np.float32)
test_labels = mnist.test.labels.astype(np.float32)
n_input = 28*28
n_labels = 10

Extracting /datasets/ud730/mnist/train-images-idx3-ubyte.gz
Extracting /datasets/ud730/mnist/train-labels-idx1-ubyte.gz
Extracting /datasets/ud730/mnist/t10k-images-idx3-ubyte.gz
Extracting /datasets/ud730/mnist/t10k-labels-idx1-ubyte.gz


In [22]:
w = weights(n_input, n_labels)
b = biases(n_labels)

In [23]:
for i in [train_features, train_labels, w, b]:
    print(sys.getsizeof(i))

172480112
2200112
56
56


In [24]:
# Features and Labels
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_labels])

In [64]:
def batches(batch_size, features, labels):
    """
    Create batches of features and labels
    :param batch_size: The batch size
    :param features: List of features
    :param labels: List of labels
    :return: Batches of (Features, Labels)
    """
    assert len(features) == len(labels)
    outout_batches = []
    sample_size = len(features)
    for start_i in range(0, sample_size, batch_size):
        end_i = start_i + batch_size
        batch = [features[start_i:end_i], labels[start_i:end_i]]
        outout_batches.append(batch)
    return outout_batches

In [65]:
# 4 Samples of features
example_features = [
    ['F11','F12','F13','F14'],
    ['F21','F22','F23','F24'],
    ['F31','F32','F33','F34'],
    ['F41','F42','F43','F44']]

# 4 Samples of labels
example_labels = [
    ['L11','L12'],
    ['L21','L22'],
    ['L31','L32'],
    ['L41','L42']]

# PPrint prints data structures like 2d arrays, so they are easier to read
pprint(batches(3, example_features, example_labels))

[[[['F11', 'F12', 'F13', 'F14'],
   ['F21', 'F22', 'F23', 'F24'],
   ['F31', 'F32', 'F33', 'F34']],
  [['L11', 'L12'], ['L21', 'L22'], ['L31', 'L32']]],
 [[['F41', 'F42', 'F43', 'F44']], [['L41', 'L42']]]]
